In [4]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras import regularizers, callbacks

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
os.getcwd()

'C:\\Users\\DarioAlejandroLatorr\\OneDrive\\claro_20_abril_2021\\ENVIO_DARIO'

In [130]:
%matplotlib qt

In [40]:
ruta = 'Data_Forecast\DWstream.csv'
divi = {'UP':(27*4), 'DW':(38*48)}
dw_up = 'DW'
data = pd.read_csv(ruta, sep=';', usecols=['NODO','FECHA','DW'])
data = pd.pivot_table(data, index='NODO', columns='FECHA', values='DW')
# interpolar, si no es posible rellenar hacia atras sino rellenar hacia adelante
data.interpolate(method='linear',  axis=1, limit_direction='backward', inplace=True) 
data.fillna(method='ffill', axis=1, inplace=True)
data = data/divi[dw_up]

In [49]:
data.head(1)

FECHA,2020-10-05,2020-10-12,2020-10-19,2020-10-26,2020-11-02,2020-11-09,2020-11-16,2020-11-23,2020-11-30,2020-12-07,...,2021-03-08,2021-03-15,2021-03-22,2021-03-29,2021-04-05,2021-04-12,2021-04-19,2021-04-26,2021-05-03,2021-05-10
NODO,,,,,,,,,,,,,,,,,,,,,
002,0.136255,0.141792,0.146243,0.144364,0.147161,0.145632,0.146932,0.153784,0.148275,0.150435,...,0.178004,0.168959,0.171431,0.185862,0.182481,0.234019,0.207596,0.211945,0.200829,0.183998


In [111]:
data.sort_values(data.columns[-1], inplace=True)

In [216]:
# dejo tres fechas para testear y dos para predecir,
# por lo tanto n_salidas  son  5 valores
# el número de entradas lo dejo como parámetro
n_entradas = 5
n_salidas = 5
factor_valid = 0.20
data_model = data[data.columns[:-3]]

X_train, Y_train = [], []
X_valid, Y_valid = [], []
len_vector = data_model.shape[1]
# número de filas diferentes por nodo y 
# límite para el inicio del fragmento de serie que se va agregando al arreglo X
n_nodo = len_vector - n_entradas - n_salidas
n_valid = 1#int(n_nodo*factor_valid)
n_train = n_nodo - n_valid
for nodo in data_model.index:
    data_vector = data_model.loc[nodo,:].values # localizo los datos de la serie de de cada nodo
    
    for i in range(n_train+1):

        X_train.append(data_vector[i:i+n_entradas])
        Y_train.append(data_vector[i+n_entradas:i+n_entradas+n_salidas])

    for i in range(n_train+1, n_nodo+1):

        X_valid.append(data_vector[i:i+n_entradas])
        Y_valid.append(data_vector[i+n_entradas:i+n_entradas+n_salidas])
    break
X_train = np.array(X_train)
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
Y_train = np.array(Y_train)

X_valid = np.array(X_valid)
X_valid = np.reshape(X_valid,(X_valid.shape[0],X_valid.shape[1],1))
Y_valid = np.array(Y_valid)

In [218]:
X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape

((19, 5, 1), (19, 5), (1, 5, 1), (1, 5))

In [220]:
n_neu = 200
epo = 4

modelo_lstm = Sequential()
modelo_lstm.add(LSTM(units= n_neu, input_shape = (n_entradas,1)))
modelo_lstm.add(Dense(units=n_salidas))
modelo_lstm.add(Dense(units=n_salidas))
modelo_lstm.compile(optimizer='rmsprop', loss='mse')
modelo_lstm.fit(X_train, Y_train, epochs=epo, verbose=1,
                validation_data=(X_valid, Y_valid), callbacks=callbacks.EarlyStopping(patience=2),shuffle=True)

Epoch 1/4
1/1 [==============================] - 3s 3s/step - loss: 2.8725e-10 - val_loss: 6.8160e-04
Epoch 2/4
1/1 [==============================] - 0s 26ms/step - loss: 6.8198e-04 - val_loss: 6.1037e-05
Epoch 3/4
1/1 [==============================] - 0s 25ms/step - loss: 6.1059e-05 - val_loss: 4.0770e-06
Epoch 4/4
1/1 [==============================] - 0s 25ms/step - loss: 4.1273e-06 - val_loss: 1.0782e-06


In [222]:
df_predict = pd.DataFrame(modelo_lstm.predict(X_valid), index=[data_model.index[0]])
df_real = pd.DataFrame(Y_valid, index=[data_model.index[0]])


In [223]:
data_model.index

Index(['EBC1', 'EBC13B', 'EBC2D2', '2HB3B', 'IU43B', '11A3B', 'AF14A1',
       'FGA3B', '8AA1', '5EG3B',
       ...
       'SYH', 'BDR', 'CER', 'CST', 'HE21', 'NVC3B', '5T5', 'CEP', 'MVL',
       'MSO'],
      dtype='object', name='NODO', length=14214)

In [225]:
for nodo in data_model.head(1).index:
    plt.figure()
    plt.plot(np.arange(len_vector),data_model.loc[[nodo],:].values[0,:],color='g')
    plt.plot(np.arange(len_vector-n_salidas, len_vector),df_predict.loc[[nodo],:].values[0,:])
    plt.plot(np.arange(len_vector-n_salidas, len_vector),df_real.loc[[nodo],:].values[0,:], color='r')
    plt.show()

In [ ]:
lag_pred = 3
step = 7
n_neu = 10
epo = 100
dim_salida=1
serie = table['OBJECTID'].values[:-lag_pred]
 
sc = MinMaxScaler()
train_data = sc.fit_transform(serie[:-2].reshape(-1, 1))
valid_data = sc.fit_transform(serie[-2-step:-1].reshape(-1, 1))
x_test_data = sc.fit_transform(serie[-1-step:-1].reshape(-1, 1))
  
X_train, Y_train = preparar_array(train_data, step)
X_valid, Y_valid = preparar_array(valid_data, step)
X_test = preparar_array(x_test_data, step, test=True)


modelo_lstm = Sequential()
modelo_lstm.add(LSTM(units= n_neu, input_shape = (step,1), kernel_regularizer=regularizers.l1(0.01),
                        activity_regularizer=regularizers.l2(0.1), bias_regularizer=regularizers.L1L2(0.01, 0.1)
                      ))
modelo_lstm.add(Dense(units=dim_salida, kernel_regularizer=regularizers.l1(0.1),
                        activity_regularizer=regularizers.l2(0.1)))
modelo_lstm.compile(optimizer='rmsprop', loss='mse')
modelo_lstm.fit(X_train, Y_train, epochs=epo, verbose=1,
                validation_data=(X_valid, Y_valid), callbacks=callbacks.EarlyStopping(patience=2),shuffle=False)
pred_no_esc = modelo_lstm.predict(X_test)
pred = sc.inverse_transform(pred_no_esc)

In [ ]:
def preparar_array(data_transform, step, test = False):
  if not test:
    X_array, Y_array = [], []
    for i in range(step,len(data_transform)):
      X_array.append(data_transform[i-step:i,0])
      Y_array.append(data_transform[i,0])
    
    X_array = np.array(X_array)
    X_array = np.reshape(X_array,(X_array.shape[0],X_array.shape[1],1))
    Y_array = np.array(Y_array)
    array = [X_array, Y_array]
  else:
    X_test = []
    for i in range(step,len(data_transform)+1):
      X_test.append(data_transform[i-step:i,0])
    X_test = np.array(X_test)
    array = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
  return array